In [4]:
import pandas as pd
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


Import data to DF

In [6]:
train_data = []
test_data = []

# Hardcoded pad naar de hoofdmap
base_path = r"/Users/hugo/AI-Tutor/Codebase/DataMiningP2/raw_files/negative_polarity"

# Loop door de klassen (deceptive en truthful)
for class_folder in os.listdir(base_path):
    class_path = os.path.join(base_path, class_folder)

    if os.path.isdir(class_path):
        subfolders = os.listdir(class_path)
        # pak de eerste 4 folder als train set
        for subfolder in subfolders[:-1]:
            subfolder_path = os.path.join(class_path, subfolder)
            for file_name in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    # lowercase alle woorden
                    content = file.read().lower()
                    # Verwijder engelse stopwoorden
                    content = ' '.join([word for word in content.split() if word not in ENGLISH_STOP_WORDS])

                    train_data.append((content, class_folder))

        # Pak de laatste folder als testset
        test_subfolder = subfolders[-1]
        test_subfolder_path = os.path.join(class_path, test_subfolder)
        for file_name in os.listdir(test_subfolder_path):
            file_path = os.path.join(test_subfolder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                # lowercase alle woorden
                content = file.read().lower()

                # Verwijder engelse stopwoorden
                content = ' '.join([word for word in content.split() if word not in ENGLISH_STOP_WORDS])

                test_data.append((content, class_folder))

# Zet de gegevens om in een DataFrame
train_df = pd.DataFrame(train_data, columns=['text', 'label'])
test_df = pd.DataFrame(test_data, columns=['text', 'label'])


EDA

In [11]:
print(train_df.head())
print(test_df.head())
print(train_df.info())
print(test_df.info())
print(train_df.describe())
print(test_df.describe())
print(train_df['label'].value_counts())
print(test_df['label'].value_counts())


label
deceptive_from_MTurk    320
truthful_from_Web       320
Name: count, dtype: int64
label
deceptive_from_MTurk    80
truthful_from_Web       80
Name: count, dtype: int64


Split train en test variables

In [13]:
X_train = train_df['text']
y_train = train_df['label']
X_test = test_df['text']
y_test = test_df['label']

--------------------------------------------------------------

Multionomial Bayes

In [40]:

# Import chi^2 test
from sklearn.feature_selection import SelectKBest, chi2

# Feature selectie
feature_selector = SelectKBest(chi2, k=1000)  # Select top 1000 features

# Maak pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('feature_selection', feature_selector),
    ('classifier', MultinomialNB())
])

# Parameter grid
param_grid = {
    'vectorizer__max_features': [2000, 3000, 4000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'feature_selection__k': [500, 1000, 1500],
    'classifier__alpha': [0.1, 0.5, 1.0, 1.5, 2.0]
}
#GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Beste estimator
best_model = grid_search.best_estimator_

# Grbuik die voor prediction met test data
y_pred = best_model.predict(X_test)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Multinomial_Naive_Bayes ({ngram_type}): ")
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Multinomial_Naive_Bayes ({ngram_type}): 
Accuracy: 0.775
Precision: 0.7756892230576441
Recall: 0.775
F1 Score: 0.774859287054409


--------------------------------------------------------------

Logistic Regression

In [42]:
pipeline_logreg = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('feature_selection', SelectKBest(chi2, k=1000)),
    ('classifier', LogisticRegression(penalty='l1', max_iter=10000))
])

param_grid_Log = {
    'vectorizer__max_features': [2000, 3000, 4000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['saga']
}

# GridSearchCV
grid_search_logreg = GridSearchCV(estimator=pipeline_logreg,
                                  param_grid=param_grid_Log,
                                  cv=5,
                                  scoring='accuracy')

# Fit op train data
grid_search_logreg.fit(X_train, y_train)

# Beste estimator
best_model_logreg = grid_search_logreg.best_estimator_

# Prediction
y_pred_logreg = best_model_logreg.predict(X_test)

# Metrics
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)

#print resultaten
accuracy_log = accuracy_score(y_test, y_pred)
precision_log = precision_score(y_test, y_pred, average='weighted')
recall_log = recall_score(y_test, y_pred, average='weighted')
f1_log = f1_score(y_test, y_pred, average='weighted')
ngram_type = "Unigrams" if grid_search_logreg.best_params_['vectorizer__ngram_range'] == (1, 1) else "Unigrams and Bigrams"


In [43]:
print(f"Logistic Regression ({ngram_type}): ")
print(f'Accuracy: {accuracy_log}')
print(f'Precision: {precision_log}')
print(f'Recall: {recall_log}')
print(f'F1 Score: {f1_log}')

Logistic Regression (Unigrams and Bigrams): 
Accuracy: 0.775
Precision: 0.7756892230576441
Recall: 0.775
F1 Score: 0.774859287054409


--------------------------------------------------------------

Decision Tree

In [44]:
# Decision Tree met pipeline
pipeline_dt = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', DecisionTreeClassifier())
])

tuned_param_dt = {
    'vectorizer__max_features': [2000, 3000, 4000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'classifier__max_depth': [3, 5, 7, None],
    'classifier__ccp_alpha': [0.0, 0.001, 0.01]
}

# GridSearchCV
grid_search_dt = GridSearchCV(
    estimator=pipeline_dt,
    param_grid=tuned_param_dt,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

# Fit op train data
grid_search_dt.fit(X_train, y_train)

# Beste estimator
best_model_dt = grid_search_dt.best_estimator_
ngram_type = "Unigrams" if grid_search_dt.best_params_['vectorizer__ngram_range'] == (1, 1) else "Unigrams and Bigrams"
# Prediction
y_pred_dt = best_model_dt.predict(X_test)

# Metrics
accuracy_tree = accuracy_score(y_test, y_pred_dt)
precision_tree = precision_score(y_test, y_pred_dt, average='weighted')
recall_tree = recall_score(y_test, y_pred_dt, average='weighted')
f1_tree = f1_score(y_test, y_pred_dt, average='weighted')

print(f"Decision Tree ({ngram_type}): ")
print(f'Accuracy: {accuracy_tree}')
print(f'Precision: {precision_tree}')
print(f'Recall: {recall_tree}')
print(f'F1 Score: {f1_tree}')


Decision Tree (Unigrams and Bigrams): 
Accuracy: 0.70625
Precision: 0.7070588235294117
Recall: 0.70625
F1 Score: 0.7059628543499512


--------------------------------------------------------------

Random Forests

In [45]:
pipeline_rf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', RandomForestClassifier())
])

tuned_param_rf = {
    'vectorizer__max_features': [2000, 3000, 4000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7, None],
    'classifier__criterion': ['entropy', 'gini'],

}

# GridSearchCV
grid_search_rf = GridSearchCV(
    estimator=pipeline_rf,
    param_grid=tuned_param_rf,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

# Fit op train data
grid_search_rf.fit(X_train, y_train)

# Beste estimator
best_model_rf = grid_search_rf.best_estimator_
ngram_type = "Unigrams" if grid_search_rf.best_params_['vectorizer__ngram_range'] == (1, 1) else "Unigrams and Bigrams"

y_pred_rf = best_model_rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(f"Random Forests ({ngram_type}): ")
print(f'Accuracy: {accuracy_rf}')
print(f'Precision: {precision_rf}')
print(f'Recall: {recall_rf}')
print(f'F1 Score: {f1_rf}')

Random Forests (Unigrams): 
Accuracy: 0.825
Precision: 0.8252032520325203
Recall: 0.825
F1 Score: 0.8249726519768714


important features uit Randomforest halen